#### pytorch搭建神经网络模型，调试超参数，观察学习率和批次大小对训练的影响。


In [ ]:
#导入包
import torch
from torchvision.datasets import KMNIST
from torchvision.transforms import ToTensor 
from torch.utils.data import DataLoader
import torch.nn as nn

# 定义超参数
LR = 0.1  #学习率调整 1.LR：0.01->Accuracy: 72.46000000000001%; 2. LR:0.005->Accuracy: 68.12% 3.LR：0.001->Accuracy: 53.400000000000006% 4.LR:0.1->Accuracy: 86.1%
epochs = 10 #训练轮次，100轮左右损失率降到了1以下，1000轮降到0.3左右，Accuracy: 76.03%
BATCH_SIZE = 32  #size:246->Accuracy: 82.49% ;size:128->Accuracy: 86.52% ; size:64->Accuracy: 88.07000000000001%;size:32->Accuracy: 88.97%


#数据集加载
train_data = KMNIST(root='./kmnist_data', train=True, download=True, transform=ToTensor())
test_data = KMNIST(root='./kmnist_data', train=False, download=True, transform=ToTensor())

#数据分批次处理
train_dl = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True) # shuffle：随机打乱数据


# 模型创建 
# 隐藏层仅1层，使用ReLU函数
model = nn.Sequential(
    nn.Linear(784, 128),  #输入层
    nn.ReLU(),     #隐藏层
    nn.Linear(128, 10)  #输出层
)


In [24]:
#损失函数和优化器
loss_fn = nn.CrossEntropyLoss() #交叉熵损失函数

#优化器（模型参数更新）
optimizer = torch.optim.SGD(model.parameters(), lr=LR) #SGD优化器

###加上训练轮次
# 训练加速，数据加载器，分批次
for epoch in range(epochs):
    for data, target in train_dl:
        #前向运算
        output = model(data.reshape(-1, 784))
        #计算损失
        loss = loss_fn(output, target) #计算梯度
        #反向传播
        optimizer.zero_grad() #所有参数梯度清零
        loss.backward()  #计算梯度（参数.grad）
        optimizer.step() #更新参数
    print(f'Epoch:{epoch}, Loss:{loss.item()}')

Epoch:0, Loss:0.11257815361022949
Epoch:1, Loss:0.2604348361492157
Epoch:2, Loss:0.08711791783571243
Epoch:3, Loss:0.14682842791080475
Epoch:4, Loss:0.0807507261633873
Epoch:5, Loss:0.06420960277318954
Epoch:6, Loss:0.08008219301700592
Epoch:7, Loss:0.020526552572846413
Epoch:8, Loss:0.10468575358390808
Epoch:9, Loss:0.021662214770913124


In [25]:
# 测试
test_dl = DataLoader(test_data, batch_size=BATCH_SIZE)

correct = 0
total = 0

with torch.no_grad(): #不计算梯度
    for data, target in test_dl:
        output = model(data.reshape(-1, 784))
        _, predicted = torch.max(output, 1) #返回第一个维度的最大值张量
        total += target.size(0) #返回张量的样本量
        correct += (predicted == target).sum().item()
print(f'Accuracy: {correct/total*100}%')

Accuracy: 88.97%
